# **CSE 475: Semi-Supervised Object Detection (Pseudo-Labeling)**

## **1. Objective**
This notebook implements **Semi-Supervised Learning (SSL)** for object detection, specifically focusing on License Plate Recognition. This project evaluates how unlabeled data can improve object detection performance using the **Pseudo-Labeling** technique.

**Key Tasks:**
1.  **Unlabeled Data Simulation:** Simulate a data-scarce environment by hiding labels for **80%** of the training data, retaining only **20%** as labeled.
2.  **Baseline Training:** Train a **YOLOv12** backbone on the limited (20%) labeled subset to establish a baseline performance.
3.  **Pseudo-Labeling (Teacher-Student Loop):**
    * Use the baseline model (Teacher) to predict labels on the 80% unlabeled data.
    * Filter predictions based on a high confidence threshold.
    * Combine original labeled data with the new "pseudo-labeled" data.
4.  **Retraining:** Train a final model (Student) on this combined dataset.
5.  **Comparison:** Evaluate the performance gain (`mAP@0.5` and `mAP@0.5:0.95`) achieved by the Semi-Supervised model against the baseline.




---

## **2. Dataset Overview**
The project utilizes the **License Plate Recognition Dataset** hosted on Roboflow Universe. This dataset is designed for Automatic Number Plate Recognition (ANPR) systems and supports single-class detection.

* **Class:** `0 -> License Plate`
* **Format:** YOLO format (`<class_id> <x_center> <y_center> <width> <height>`)
* **Diversity:** The dataset includes images of cars, motorcycles, buses, and trucks captured in diverse environments (streets, parking lots, highways).
* **Challenges:** The data includes challenging real-world scenarios such as motion blur, nighttime visibility, sun glare, and partial occlusions.

### **Data Distribution**
The original dataset is split as follows:
* **🟢 Train:** 7,057 images
* **🔵 Validate:** 2,048 images
* **🟠 Test:** 1,020 images

### **Experimental Split (Unlabeled Simulation)**
To adhere to the assignment constraints and simulate a semi-supervised learning scenario, the **Train** set (7,057 images) is randomly split as follows:
* **Labeled Set (20%):** **~1,411 images**. Used for training the baseline model and initial supervision.
* **Unlabeled Set (80%):** **~5,646 images**. Labels are discarded/hidden. These images are used for the Pseudo-Labeling inference step.

---

## **3. Baseline Detector Summary**
* **Model Architecture:** YOLOv12n (Nano) - Ultralytics
* **Reason for Selection:** This was the best-performing object detector from **Assignment 1**, chosen for its high accuracy (mAP) and ultra-fast inference speed suitable for real-time ANPR on edge devices.
* **Assignment 1 Performance:**
    * **Backbone:** YOLOv12n (csp-darknet)
    * **Previous mAP:** `mAP@0.5`: 0.9765 | `mAP@0.5:0.95`: 0.7307

---

In [1]:
# Install necessary libraries
!pip install ultralytics roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━

## **4. Data Acquisition & Splitting**
We download the dataset using the Roboflow API. To comply with the assignment, we then physically split the data into two folders:
1.  `datasets/labeled_data`: Contains images and labels for the 20% subset.
2.  `datasets/unlabeled_data`: Contains only images for the 80% subset (labels are stripped).

In [2]:
import os
import shutil
import random
import glob
import yaml
from roboflow import Roboflow
from tqdm.notebook import tqdm

# --- 1. DOWNLOAD DATASET ---
rf = Roboflow(api_key="SwgvVj4W6ABZJbJVXoMv")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(11)
dataset = version.download("yolov12")

# --- 2. CONFIGURE SPLIT ---
INPUT_DATASET_DIR = dataset.location
BASE_OUTPUT_DIR = '/kaggle/working/datasets'
LABELED_DIR = os.path.join(BASE_OUTPUT_DIR, 'labeled_data')
UNLABELED_DIR = os.path.join(BASE_OUTPUT_DIR, 'unlabeled_data')

LABELED_RATIO = 0.20 # 20% Labeled
SEED = 42

# --- 3. EXECUTE SPLIT ---
def split_data():
    # Create Directories
    for split in ['train', 'valid', 'test']:
        os.makedirs(os.path.join(LABELED_DIR, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(LABELED_DIR, split, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(UNLABELED_DIR, 'images'), exist_ok=True)
    os.makedirs(os.path.join(UNLABELED_DIR, 'labels'), exist_ok=True)

    # Shuffle & Split
    random.seed(SEED)
    train_images = glob.glob(os.path.join(INPUT_DATASET_DIR, 'train', 'images', '*.*'))
    random.shuffle(train_images)
    
    split_idx = int(len(train_images) * LABELED_RATIO)
    labeled_set = train_images[:split_idx]
    unlabeled_set = train_images[split_idx:]
    
    print(f"📊 Splitting Data: {len(labeled_set)} Labeled | {len(unlabeled_set)} Unlabeled")

    # Copy Labeled Data (Images + Labels)
    print("   -> Copying Labeled Set...")
    for img in tqdm(labeled_set):
        shutil.copy(img, os.path.join(LABELED_DIR, 'train', 'images', os.path.basename(img)))
        # Copy corresponding label
        base_name = os.path.basename(img).rsplit('.', 1)[0]
        src_lbl = os.path.join(os.path.dirname(img).replace('images', 'labels'), base_name + '.txt')
        if os.path.exists(src_lbl):
            shutil.copy(src_lbl, os.path.join(LABELED_DIR, 'train', 'labels', base_name + '.txt'))

    # Copy Unlabeled Data (Images Only)
    print("   -> Copying Unlabeled Set...")
    for img in tqdm(unlabeled_set):
        shutil.copy(img, os.path.join(UNLABELED_DIR, 'images', os.path.basename(img)))

    # Copy Val/Test (Keep 100% Intact)
    print("   -> Copying Validation/Test Sets...")
    for split in ['valid', 'test']:
        src_img_dir = os.path.join(INPUT_DATASET_DIR, split, 'images')
        src_lbl_dir = os.path.join(INPUT_DATASET_DIR, split, 'labels')
        if os.path.exists(src_img_dir):
            for f in os.listdir(src_img_dir):
                shutil.copy(os.path.join(src_img_dir, f), os.path.join(LABELED_DIR, split, 'images', f))
            for f in os.listdir(src_lbl_dir):
                shutil.copy(os.path.join(src_lbl_dir, f), os.path.join(LABELED_DIR, split, 'labels', f))

split_data()

# --- 4. CREATE YAML CONFIG ---
with open(os.path.join(dataset.location, 'data.yaml'), 'r') as f: 
    data_conf = yaml.safe_load(f)

new_yaml = {
    'path': LABELED_DIR,
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': data_conf['nc'],
    'names': data_conf['names']
}

yaml_path = '/kaggle/working/datasets/labeled_data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(new_yaml, f, sort_keys=False)

print(f"✅ YAML Configuration saved at: {yaml_path}")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to License-Plate-Recognition-11 in yolov12:: 100%|██████████| 20262/20262 [00:02<00:00, 8348.59it/s]


📊 Splitting Data: 1411 Labeled | 5646 Unlabeled
   -> Copying Labeled Set...


  0%|          | 0/1411 [00:00<?, ?it/s]

   -> Copying Unlabeled Set...


  0%|          | 0/5646 [00:00<?, ?it/s]

   -> Copying Validation/Test Sets...
✅ YAML Configuration saved at: /kaggle/working/datasets/labeled_data.yaml


## **5. Phase 1: Baseline Training (Teacher)**
We train a YOLOv12 Nano model on the 20% labeled data. This serves as our "Teacher" model.

In [3]:
# Fix Numpy Version Conflict
!pip install "numpy<2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 61.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 32.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but yo

In [4]:
# --- PHASE 1: TRAIN TEACHER (BASELINE) ---


print("🚀 Starting Phase 1: Teacher Training...")

!yolo train \
    model=yolo12n.pt \
    data=/kaggle/working/datasets/labeled_data.yaml \
    epochs=50 \
    imgsz=640 \
    batch=110 \
    project=runs/train \
    name=baseline_teacher \
    device=0,1 \
    workers=2 \
    exist_ok=True

🚀 Starting Phase 1: Teacher Training...
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.233 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=110, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/datasets/labeled_data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.

## **6. Phase 2: Pseudo-Labeling Inference**
The Teacher model predicts labels for the 80% unlabeled data.
* **Threshold:** `0.75` (We only accept high-confidence predictions).
* **Method:** We use the CLI `predict` mode to ensure stability and avoid memory issues on Kaggle.

In [5]:
import os
import shutil

# --- PHASE 2: PSEUDO-LABELING (SILENT MODE) ---
print("🚀 Starting Phase 2: Generating Pseudo-Labels...")
print("   (Output is being saved to 'inference_log.txt' to prevent notebook clogging...)")
print("   ⏳ This may take a few minutes. Please wait.")

# 1. Define Paths
TEACHER_WEIGHTS = '/kaggle/working/runs/train/baseline_teacher/weights/best.pt'
UNLABELED_SOURCE = '/kaggle/working/datasets/unlabeled_data/images'

# 2. Run Inference via CLI (Silent)
# We add '> inference_log.txt' at the end to hide the output
!yolo predict \
    model={TEACHER_WEIGHTS} \
    source={UNLABELED_SOURCE} \
    conf=0.75 \
    save_txt=True \
    save_conf=False \
    save=False \
    project=runs/detect \
    name=pseudo_labels \
    exist_ok=True > inference_log.txt

# 3. Move Generated Labels to the Dataset Folder
print("\n📦 Inference Done! Moving labels...")
PREDICT_LABEL_DIR = 'runs/detect/pseudo_labels/labels'
DEST_LABEL_DIR = '/kaggle/working/datasets/unlabeled_data/labels'

# Ensure destination exists
os.makedirs(DEST_LABEL_DIR, exist_ok=True)

if os.path.exists(PREDICT_LABEL_DIR):
    files = os.listdir(PREDICT_LABEL_DIR)
    count = 0
    for f in files:
        if f.endswith('.txt'):
            shutil.move(os.path.join(PREDICT_LABEL_DIR, f), os.path.join(DEST_LABEL_DIR, f))
            count += 1
    print(f"✅ Moved {count} pseudo-labels to {DEST_LABEL_DIR}")
else:
    print("⚠️ No labels generated! (Check 'inference_log.txt' to see what happened)")

🚀 Starting Phase 2: Generating Pseudo-Labels...
   (Output is being saved to 'inference_log.txt' to prevent notebook clogging...)
   ⏳ This may take a few minutes. Please wait.

📦 Inference Done! Moving labels...
✅ Moved 5020 pseudo-labels to /kaggle/working/datasets/unlabeled_data/labels


## **7. Phase 3: Student Training**
We combine the original labeled data with the newly generated pseudo-labeled data and train a fresh Student model.

In [6]:
# 1. Create Combined YAML
combined_yaml = {
    'path': '/kaggle/working/datasets', 
    'train': [
        'labeled_data/train/images',  # Original 20%
        'unlabeled_data/images'       # The 80% (Pseudo-Labeled)
    ],
    'val': 'labeled_data/valid/images',
    'test': 'labeled_data/test/images',
    'nc': 1,
    'names': ['license_plate']
}

with open('/kaggle/working/datasets/combined_data.yaml', 'w') as f:
    yaml.dump(combined_yaml, f, sort_keys=False)

# 2. Train Student Model
print("🚀 Starting Phase 3: Student Training...")
!yolo train \
    model=yolo12n.pt \
    data=/kaggle/working/datasets/combined_data.yaml \
    epochs=50 \
    imgsz=640 \
    batch=110 \
    project=runs/train \
    name=student_ssl \
    device=0,1 \
    workers=2 \
    exist_ok=True

🚀 Starting Phase 3: Student Training...
Ultralytics 8.3.233 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=110, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/datasets/combined_data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=s

In [7]:
import pandas as pd
import os
from ultralytics import YOLO
from IPython.display import display, Markdown

# --- FINAL TEST SET EVALUATION ---
print("📊 Starting Final Evaluation on TEST SET (Held-out data)...")

# 1. Define Paths & Reference Scores
teacher_weight = '/kaggle/working/runs/train/baseline_teacher/weights/best.pt'
student_weight = '/kaggle/working/runs/train/student_ssl/weights/best.pt'
test_config = '/kaggle/working/datasets/labeled_data.yaml' 

# Assignment 1 Reference Scores (Hardcoded from your input)
a1_map50 = 0.9765
a1_map95 = 0.7307

# 2. Run Validation on 'split=test'
print("   -> Evaluating Teacher (Phase 1 Baseline)...")
try:
    model_t = YOLO(teacher_weight)
    # split='test' ensures we evaluate on the Test set, not Validation set
    metrics_t = model_t.val(data=test_config, split='test', verbose=False, plots=False)
    t_map50 = metrics_t.box.map50
    t_map95 = metrics_t.box.map
except Exception as e:
    print(f"⚠️ Could not evaluate Teacher: {e}")
    t_map50, t_map95 = 0, 0

print("   -> Evaluating Student (Phase 3 Semi-Supervised)...")
try:
    model_s = YOLO(student_weight)
    metrics_s = model_s.val(data=test_config, split='test', verbose=False, plots=False)
    s_map50 = metrics_s.box.map50
    s_map95 = metrics_s.box.map
except Exception as e:
    print(f"⚠️ Could not evaluate Student: {e}")
    s_map50, s_map95 = 0, 0

# 3. Generate Report & Calculations
# Calculate improvement over Teacher (Baseline)
delta_teacher_50 = s_map50 - t_map50
delta_teacher_95 = s_map95 - t_map95

# Formatting signs (add + if positive)
sign_t50 = "+" if delta_teacher_50 > 0 else ""
sign_t95 = "+" if delta_teacher_95 > 0 else ""

# Calculate gap to Assignment 1 (Full Supervised Reference)
delta_a1_50 = s_map50 - a1_map50

report_md = "## **8. Performance Comparison (Test Set)**\n\n"
report_md += "| Model | Training Data | `mAP@0.5` | `mAP@0.5:0.95` |\n"
report_md += "| :--- | :--- | :--- | :--- |\n"
report_md += f"| **Assignment 1 (Reference)** | Full Data (Previous) | *{a1_map50:.4f}* | *{a1_map95:.4f}* |\n"
report_md += f"| **Phase 1 Baseline (Teacher)** | 20% Labeled | {t_map50:.4f} | {t_map95:.4f} |\n"
report_md += f"| **Phase 3 SSL (Student)** | 20% Labeled + Pseudo | **{s_map50:.4f}** | **{s_map95:.4f}** |\n"
report_md += f"| **Improvement (vs Phase 1)** | - | **{sign_t50}{delta_teacher_50:.4f}** | **{sign_t95}{delta_teacher_95:.4f}** |\n\n"

report_md += "### **Discussion**\n"
if delta_teacher_50 > 0:
    report_md += f"The Semi-Supervised Learning (Student) model successfully outperformed the Phase 1 Baseline (Teacher) by **{delta_teacher_50:.4f} `mAP@0.5`**. "
    report_md += "This confirms that the pseudo-labeling pipeline effectively leveraged the 80% unlabeled data to learn robust features, even without manual annotations. "
else:
    report_md += f"The Student model performance was comparable to the Teacher ({delta_teacher_50:.4f}). This suggests that while the model remained stable, the pseudo-labels might have contained noise that prevented significant gains. "

report_md += f"When compared to the Assignment 1 Reference (trained on full data), the Semi-Supervised model has closed the gap by achieving **{s_map50:.4f}**, which is **{delta_a1_50:.4f}** away from the fully supervised performance. "
report_md += "This demonstrates that SSL is a viable strategy for data-scarce environments, recovering a significant portion of the performance that would typically require full manual annotation."

report_md += "\n\n## **9. Conclusion**\n"
report_md += "We successfully implemented a Semi-Supervised pipeline using YOLOv12. By training a Teacher on limited data (20%) and generating pseudo-labels for the remaining 80%, we demonstrated that unlabeled data can be used to improve detection accuracy compared to a strictly supervised baseline."

report_md += "\n\n## **10. References**\n"
report_md += "1. **Dataset:** Roboflow Universe Projects. (2025). [License Plate Recognition Dataset](https://universe.roboflow.com/roboflow-universe-projects/license-plate-recognition-rxg4e).\n"
report_md += "2. **Libraries:** Ultralytics YOLOv12, PyTorch.\n"
report_md += "3. **Method:** Self-Training with Pseudo-Labeling.\n"

display(Markdown(report_md))

📊 Starting Final Evaluation on TEST SET (Held-out data)...
   -> Evaluating Teacher (Phase 1 Baseline)...
Ultralytics 8.3.233 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 888.8±569.2 MB/s, size: 30.4 KB)
val: Scanning /kaggle/working/datasets/labeled_data/test/labels... 1020 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1020/1020 1.5Kit/s 0.7s
val: New cache created: /kaggle/working/datasets/labeled_data/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 64/64 10.5it/s 6.1s
                   all       1020       1085      0.986      0.921       0.96      0.688
Speed: 0.5ms preprocess, 3.7ms inference, 0.0ms loss, 0.6ms postprocess per image
   -> Evaluating Student (Phase 3 Semi-Supervised)...
Ultralytics 8.3.233 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (

## **8. Performance Comparison (Test Set)**

| Model | Training Data | `mAP@0.5` | `mAP@0.5:0.95` |
| :--- | :--- | :--- | :--- |
| **Assignment 1 (Reference)** | Full Data (Previous) | *0.9765* | *0.7307* |
| **Phase 1 Baseline (Teacher)** | 20% Labeled | 0.9604 | 0.6879 |
| **Phase 3 SSL (Student)** | 20% Labeled + Pseudo | **0.9656** | **0.7103** |
| **Improvement (vs Phase 1)** | - | **+0.0052** | **+0.0223** |

### **Discussion**
The Semi-Supervised Learning (Student) model successfully outperformed the Phase 1 Baseline (Teacher) by **0.0052 `mAP@0.5`**. This confirms that the pseudo-labeling pipeline effectively leveraged the 80% unlabeled data to learn robust features, even without manual annotations. When compared to the Assignment 1 Reference (trained on full data), the Semi-Supervised model has closed the gap by achieving **0.9656**, which is **-0.0109** away from the fully supervised performance. This demonstrates that SSL is a viable strategy for data-scarce environments, recovering a significant portion of the performance that would typically require full manual annotation.

## **9. Conclusion**
We successfully implemented a Semi-Supervised pipeline using YOLOv12. By training a Teacher on limited data (20%) and generating pseudo-labels for the remaining 80%, we demonstrated that unlabeled data can be used to improve detection accuracy compared to a strictly supervised baseline.

## **10. References**
1. **Dataset:** Roboflow Universe Projects. (2025). [License Plate Recognition Dataset](https://universe.roboflow.com/roboflow-universe-projects/license-plate-recognition-rxg4e).
2. **Libraries:** Ultralytics YOLOv12, PyTorch.
3. **Method:** Self-Training with Pseudo-Labeling.
